In [1]:
import numpy as np
# import pyvista as pv
from collections import defaultdict, deque
import copy

# ----------- Normal vectors estimation functions -----------

def compute_face_normals_and_areas(mesh):
    """
    Computes normal and area for each triangle face in the mesh.

    Returns:
        normals (np.ndarray): An (F, 3) array of unit normal vectors for each face.
        areas (np.ndarray): A (F,) array of areas for each face.

    Method:
        - Extract vertex indices for each triangle face.
        - Compute two edge vectors of the triangle.
        - Use cross product to get the face normal (not normalized).
        - Normalize to get unit normals.
        - Compute triangle area as half of the magnitude of the cross product.
    """
    faces = mesh.faces.reshape((-1, 4))[:, 1:]
    v0 = mesh.points[faces[:, 0]]
    v1 = mesh.points[faces[:, 1]]
    v2 = mesh.points[faces[:, 2]]
    
    cross = np.cross(v1 - v0, v2 - v0)
    areas = 0.5 * np.linalg.norm(cross, axis=1)
    normals = cross / np.maximum(np.linalg.norm(cross, axis=1, keepdims=True), 1e-8)
    return normals, areas

def build_vertex_to_faces_map(mesh):
    """
    Builds a mapping from each vertex index to the list of face indices that include that vertex.

    The same with pyvista's mesh.point_cell_ids()
    Returns:
        defaultdict(list): A dictionary where each key is a vertex index, and the value is 
        a list of face indices (from mesh.faces) that contain the vertex.

    Example:
        {
            0: {1, 5, 10},  # vertex 0 is part of faces 1, 5, and 10
            1: {0, 2},      # vertex 1 is part of faces 0 and 2
            ...
        }    
    """
    vertex_faces = defaultdict(list)
    faces = mesh.faces.reshape((-1, 4))[:, 1:]
    for i, (v1, v2, v3) in enumerate(faces):
        vertex_faces[v1].append(i)
        vertex_faces[v2].append(i)
        vertex_faces[v3].append(i)
    return vertex_faces

def compute_area_weighted_vertex_normals(mesh):
    """
    Computes unit vertex normals by averaging adjacent face normals, weighted by face area.

    Returns:
        vertex_normals (np.ndarray): An (N, 3) array of unit normal vectors for each vertex.

    Method:
        1. Compute face normals and face areas using cross products.
        2. Build a mapping from each vertex to the list of adjacent face indices.
        3. For each vertex:
            - Retrieve adjacent faces.
            - Average their normals, weighted by face area.
            - Normalize the result to obtain a unit normal.
    """
    face_normals, face_areas = compute_face_normals_and_areas(mesh)
    vertex_faces = build_vertex_to_faces_map(mesh)
    num_vertices = mesh.points.shape[0]
    vertex_normals = np.zeros((num_vertices, 3))

    for vi in range(num_vertices):
        adjacent_faces = vertex_faces[vi]
        if not adjacent_faces:
            continue
        areas = face_areas[adjacent_faces]
        normals = face_normals[adjacent_faces]
        weighted_sum = np.sum(normals * areas[:, np.newaxis], axis=0)
        vertex_normals[vi] = weighted_sum / np.linalg.norm(weighted_sum)
    
    return vertex_normals

# -------------------------- Normal vector angle --------------

def normal_vector_angle(n_p, n_q, degrees=False):
    """
    Parameters:
    - n_p, n_q: 3D vectors (can be non-normalized).
    - degrees: If True, return angle in degrees. Otherwise, radians.

    Returns:
    - Angle between vectors
    """
    dot_product = np.dot(n_p, n_q)
    norm_p = np.linalg.norm(n_p)
    norm_q = np.linalg.norm(n_q)

    if norm_p == 0 or norm_q == 0:
        return 0.0

    cos_theta = np.clip(dot_product / (norm_p * norm_q), -1.0, 1.0)
    angle_rad = np.arccos(cos_theta)
    return np.degrees(angle_rad) if degrees else angle_rad

# ----------------------------------------------
def sorting_flat_vertices_with_flatness(normals, adjacency):
    """
    Find all flat vertices, compute their flatness (average normal angle to neighbors),
    and return them sorted by flatness (ascending).
    
    Returns:
        List of (vertex_index, flatness_value) tuples, sorted by flatness_value.
    """
    flat_vertices = []
    for vi, neighbors in adjacency.items():
        if not neighbors:
            continue
        # Check if all neighbor angles are below threshold
        angles = [normal_vector_angle(normals[vi], normals[vj]) for vj in neighbors]
        flatness = sum(angles) / len(angles)
        flat_vertices.append((vi, flatness))
        # if all(angle < threshold for angle in angles):
        #     flatness = sum(angles) / len(angles)
        #     flat_vertices.append((vi, flatness))
        
    # Sort by flatness value (ascending)
    flat_vertices.sort(key=lambda x: x[1])
    return flat_vertices

from collections import deque
import numpy as np
import copy

def cluster_surface(adj, v_normals, flat_v_sorted, weight, threshold):
    flat_v_sorted_copy = deque(flat_v_sorted)  # Faster .popleft()
    unvisited = set(range(len(v_normals)))
    clusters = []

    def compute_avg_angle(normal, neighbors):
        return np.mean([
            normal_vector_angle(normal, v_normals[nj])
            for nj in neighbors
        ])

    while unvisited:
        while flat_v_sorted_copy:
            start = flat_v_sorted_copy.popleft()
            if start in unvisited:
                break
        else:
            break  # No unvisited nodes left

        cluster = set([start])
        queue = deque([start])
        unvisited.remove(start)

        # Maintain running sum of normals for this cluster
        normals_sum = v_normals[start].copy()

        while queue:
            vi = queue.popleft()
            for vj in adj[vi]:
                if vj not in unvisited:
                    continue

                # Average angle between vj and its neighbors
                avg_angle_vi = compute_avg_angle(v_normals[vi], adj[vj])

                # Compute updated cluster normal
                cluster_normal = normals_sum / np.maximum(np.linalg.norm(normals_sum), 1e-8)

                avg_angle_cluster = compute_avg_angle(cluster_normal, adj[vj])
                proximity = weight * avg_angle_vi + (1 - weight) * avg_angle_cluster

                if proximity < threshold:
                    cluster.add(vj)
                    queue.append(vj)
                    unvisited.remove(vj)
                    normals_sum += v_normals[vj]  # Update running normal sum

        clusters.append(cluster)

    return clusters


def merge_noise_faces(clusters, vertex_normals, adjacency, K=10):
    """
    Merge small (noise) clusters into neighboring real clusters based on normal similarity,
    while avoiding merging adjacent noise clusters together.

    Parameters:
        clusters (List[Set[int]]): List of vertex index sets (each cluster).
        vertex_normals (np.ndarray): (N, 3) array of vertex normals.
        adjacency (dict): {vertex_index: set(neighboring vertex indices)}.
        K (int): Max vertex count for a cluster to be considered noise.

    Returns:
        List[Set[int]]: Updated clusters with noise surfaces merged into real surfaces.
    """
    # Separate clusters
    real_clusters = [c for c in clusters if len(c) >= K]
    noise_clusters = [c for c in clusters if len(c) < K]

    # Precompute normals
    def cluster_avg_normal(cluster):
        normals = [vertex_normals[vi] for vi in cluster]
        summed = np.sum(normals, axis=0)
        return summed / np.linalg.norm(summed) if np.linalg.norm(summed) > 0 else summed

    real_normals = [cluster_avg_normal(c) for c in real_clusters]
    noise_normals = [cluster_avg_normal(c) for c in noise_clusters]

    # Create index-to-cluster maps for fast lookup
    vertex_to_real_cluster = {}
    for idx, cluster in enumerate(real_clusters):
        for v in cluster:
            vertex_to_real_cluster[v] = idx

    merged_indices = set()  # track merged noise cluster indices

    for ni, noise in enumerate(noise_clusters):
        if ni in merged_indices:
            continue
        noise_normal = noise_normals[ni]

        # Find adjacent real clusters only
        neighbor_real_candidates = set()
        for vi in noise:
            for vj in adjacency[vi]:
                if vj in vertex_to_real_cluster:
                    neighbor_real_candidates.add(vertex_to_real_cluster[vj])

        # Find the best matching real cluster by normal vector angle
        best_real_idx = None
        best_angle = float('inf')

        for r_idx in neighbor_real_candidates:
            angle = normal_vector_angle(noise_normal, real_normals[r_idx])
            if angle < best_angle:
                best_real_idx = r_idx
                best_angle = angle

        # Merge into best-matching real cluster
        if best_real_idx is not None:
            real_clusters[best_real_idx].update(noise)
            merged_indices.add(ni)
        # else: discard the noise cluster (could also be kept separately)

    return real_clusters


'''
    This code implement the Laplace smoothing algorithm mentioned in the section 2 of the paper 
    "Robust surface segmentation and edge feature lines extraction from fractured
    ragments of relics" 
'''


# ---------- Laplacian Smoothing Functions ----------

def build_vertex_adjacency(faces):
    adjacency = defaultdict(set)
    for i, j, k in faces:
        adjacency[i].update((j, k))
        adjacency[j].update((i, k))
        adjacency[k].update((i, j))
    return adjacency

def weighted_laplacian_smoothing(vertices, faces, iterations=1):
    vertices = vertices.copy()
    adjacency = build_vertex_adjacency(faces)

    for _ in range(iterations):
        new_vertices = vertices.copy()
        for i in range(vertices.shape[0]):
            neighbors = list(adjacency[i])
            if not neighbors:
                continue

            neighbors_coords = vertices[neighbors]  # shape: (N_neighbors, 3)
            diffs = neighbors_coords - vertices[i]  # shape: (N_neighbors, 3)
            dists = np.linalg.norm(diffs, axis=1) + 1e-8  # avoid zero division
            weights = 1.0 / dists
            weights /= np.sum(weights)

            weighted_sum = np.dot(weights, diffs)  # shape: (3,)
            new_vertices[i] += weighted_sum

        vertices = new_vertices

    return vertices

def compute_mesh_smoothness(vertices, faces): # To see how much a mesh is smooth
    adjacency = build_vertex_adjacency(faces)
    smoothness_vals = []

    for i in range(len(vertices)):
        neighbors = list(adjacency[i])
        if not neighbors:
            continue
        avg_neighbor = np.mean(vertices[neighbors], axis=0)
        smoothness = np.linalg.norm(vertices[i] - avg_neighbor)
        smoothness_vals.append(smoothness)

    return np.mean(smoothness_vals)

In [1]:
!pip install open3d

ERROR: Could not find a version that satisfies the requirement open3d (from versions: none)
ERROR: No matching distribution found for open3d


In [2]:
def convert_to_builtin_int(obj):
    if isinstance(obj, dict):
        return {convert_to_builtin_int(k): convert_to_builtin_int(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_builtin_int(i) for i in obj]
    elif isinstance(obj, set):
        return {convert_to_builtin_int(i) for i in obj}
    elif isinstance(obj, tuple):
        return tuple(convert_to_builtin_int(i) for i in obj)
    elif isinstance(obj, np.integer):  # Catch np.int64, np.int32, ...
        return int(obj)
    else:
        return obj


In [3]:
import os
import pyvista as pv

mesh = pv.read('D:/Learn_and_Study/USTH/Bachelor/3D_Project/CG_dataset/brick_part01.obj')

In [4]:
faces_1D = mesh.faces
faces_nD = faces_1D.reshape((-1, 4))[:, 1:]

smoothed_vertices = weighted_laplacian_smoothing(mesh.points, faces_nD, 3)
smoothed_mesh = pv.PolyData(smoothed_vertices, faces_1D)

import gc
del mesh
del smoothed_vertices
gc.collect()

23

In [5]:
import gc

vertices = smoothed_mesh.points 
faces_1D = smoothed_mesh.faces
faces_nD = smoothed_mesh.faces.reshape((-1, 4))[:, 1:]

del faces_1D
gc.collect()

0

In [6]:
adj = build_vertex_adjacency(faces_nD)
adj = convert_to_builtin_int(adj)

v_normals = compute_area_weighted_vertex_normals(smoothed_mesh)

flat_v_sorted = sorting_flat_vertices_with_flatness(v_normals, adj)
flat_v_sorted = [vertex for vertex, flatness in flat_v_sorted]
flat_v_sorted = convert_to_builtin_int(flat_v_sorted)

### Test cluster_surface()

In [7]:
clusters = cluster_surface(adj, v_normals, flat_v_sorted, 0.5, 0.5)

### Test merge_noise_face() to remove noise faces

In [8]:
merged = merge_noise_faces(clusters, v_normals, adj, K=10)

In [20]:
colors = [
    'red', 'green', 'blue', 'yellow', 'magenta', 'cyan',
    'orange', 'lime', 'deeppink', 'deepskyblue', 'gold',
    'indigo', 'teal', 'crimson', 'mediumvioletred',
    'chartreuse', 'orangered', 'darkturquoise',
    'slateblue', 'darkgoldenrod'
]

plotter = pv.Plotter(shape=(1, 3))

plotter.subplot(0, 0)
plotter.add_title("Original")
plotter.add_mesh(smoothed_mesh, show_edges=True)

plotter.subplot(0, 1)
plotter.add_title("Clusters")
plotter.add_mesh(smoothed_mesh, show_edges=True)
for i, cluster in enumerate(clusters):
    cluster_indices = list(cluster)
    color = colors[i % len(colors)]
    plotter.add_mesh(smoothed_mesh.points[cluster_indices], color=color, point_size=10)


plotter.subplot(0, 2)
plotter.add_title("Merged")
plotter.add_mesh(smoothed_mesh, show_edges=True)
for i, cluster in enumerate(merged):
    merged_indices = list(cluster)
    color = colors[i % len(colors)]
    plotter.add_mesh(smoothed_mesh.points[merged_indices], color=color, point_size=10)

plotter.link_views()
plotter.show()

import gc
del colors, color, i, plotter, cluster_indices, merged_indices, cluster
gc.collect()


Widget(value='<iframe src="http://localhost:52339/index.html?ui=P_0x17900b5a490_3&reconnect=auto" class="pyvis…

0

### Test surface differentiation

In [ ]:
original_indices = list(merged[0])[:50] + list(merged[2])[:50]
fractured_indices = list(merged[5])[:50] + list(merged[4])[:50]

In [40]:
training_original_indices = np.array([32768, 32770, 32771, 32772, 32792, 32795, 32829, 32833, 32838, 32840, 32848, 32875, 32878, 32886, 185, 32974, 32976, 32978, 32988, 32991, 32996, 33002, 33005, 33008, 33010, 33022, 33025, 33027, 33034, 33058, 33061, 33064, 33067, 33091, 33093, 33095, 33097, 33100, 33110, 33114, 33125, 33208, 33210, 33214, 33218, 33224, 33227, 33228, 463, 465, 16384, 0, 1, 16387, 5, 7, 8, 1630, 11, 15, 18, 20, 21, 23, 26, 27, 28, 32, 37, 38, 40, 8234, 42, 46, 8239, 8244, 53, 8250, 59, 60, 62, 66, 67, 68, 8263, 71, 73, 74, 76, 77, 8270, 78, 79, 8278, 86, 8282, 93, 95, 96, 8293])
training_fractured_indices = np.array([32774, 32775, 32777, 32778, 32782, 32783, 32785, 32787, 32788, 32789, 32791, 32796, 32798, 32799, 32801, 32802, 32803, 32804, 32806, 32807, 32808, 32809, 32812, 32814, 32815, 32819, 32822, 32823, 32824, 32827, 32830, 32834, 32836, 32841, 32843, 32845, 32847, 32850, 32853, 32857, 32858, 32859, 32861, 32863, 32864, 32867, 32868, 32869, 32876, 32880, 32769, 32773, 32776, 32779, 32780, 32781, 32784, 32786, 32790, 32793, 32794, 32797, 32805, 32810, 32813, 32816, 32817, 32818, 32820, 32821, 32825, 32826, 32828, 32831, 32832, 32835, 32837, 32839, 32842, 32844, 32846, 32849, 32851, 32852, 32854, 32855, 32856, 32860, 32862, 32865, 32866, 32870, 32871, 32872, 32873, 32874, 32877, 32879, 32881, 32885])

test_original_indices = np.array([32770, 32773, 32778, 32781, 32786, 32790, 32791, 32792, 32798, 32799, 32800, 32880, 32933, 32938, 32946, 32952, 33004, 33007, 33010, 33014, 33016, 33019, 33021, 33039, 33044, 33054, 33056, 33060, 33070, 33094, 33097, 33125, 33135, 33153, 33156, 33161, 33163, 33189, 33191, 33194, 33198, 33199, 33202, 33205, 33208, 33209, 33212, 33214, 33216, 33218, 32769, 24578, 24580, 20485, 18438, 32774, 32777, 20493, 20495, 32783, 10255, 20498, 18451, 10258, 10260, 20502, 22550, 20504, 10263, 18458, 24603, 20507, 18460, 20510, 10266, 10270, 18466, 28710, 18471, 18473, 24618, 18474, 20524, 28714, 18478, 28718, 30769, 28722, 18485, 18486, 30774, 18488, 12343, 18490, 28730, 18492, 24637, 28733, 24640, 28736])
test_fractured_indices = np.array([6513, 32772, 32775, 32776, 32779, 32780, 32782, 32784, 32785, 32787, 32788, 32789, 32795, 32801, 34, 32803, 33, 32805, 32806, 32807, 32808, 32809, 32810, 32811, 32812, 32813, 32814, 32815, 32816, 32817, 32818, 32819, 32820, 32821, 32822, 32823, 32824, 32825, 32826, 32827, 32828, 32829, 32830, 32831, 32832, 32833, 32834, 32835, 32836, 32837, 32768, 32771, 32793, 32794, 32796, 32843, 32845, 32846, 32847, 32848, 32849, 32850, 32851, 32852, 32853, 32854, 32855, 32856, 32857, 32858, 32859, 32861, 32864, 32868, 32869, 32870, 32872, 32874, 32875, 32876, 32878, 32882, 32883, 32884, 32885, 32886, 32888, 32889, 32891, 32894, 32895, 32896, 32897, 32898, 32899, 32900, 32901, 32902, 32903, 32904])

In [41]:
def compute_local_bending_energy(vertices, vertex_normals, adjacency):
    """
    Computes local bending energy e_k(p) for each vertex p.

    Parameters:
        vertices (np.ndarray): (N, 3) array of vertex positions.
        vertex_normals (np.ndarray): (N, 3) array of vertex normals.
        adjacency (dict): {vertex_index: set of neighbor vertex indices}

    Returns:
        e_k (np.ndarray): (N,) array of local bending energy per vertex.
    """
    N = len(vertices)
    e_k = np.zeros(N)

    for p, neighbors in adjacency.items():
        if not neighbors:
            continue

        v_p = vertices[p]
        n_p = vertex_normals[p]

        v_neighbors = vertices[list(neighbors)]
        n_neighbors = vertex_normals[list(neighbors)]

        # Vector differences
        norm_diffs = np.sum((n_p - n_neighbors) ** 2, axis=1)
        geom_dists = np.sum((v_p - v_neighbors) ** 2, axis=1) + 1e-8  # prevent divide by 0

        e_k[p] = np.mean(norm_diffs / geom_dists)

    return e_k

from scipy.spatial import cKDTree

from scipy.spatial import cKDTree
import numpy as np

def compute_integrated_bending_energy_fast(e_k, vertices, radius):
    """
    RAM-efficient version of computing e_{k,r}(p).
    Parameters:
        e_k (np.ndarray): (N,) local bending energy per vertex.
        vertices (np.ndarray): (N, 3) coordinates.
        radius (float): Radius for neighborhood averaging.
    Returns:
        e_kr (np.ndarray): (N,) integrated bending energy.
    """
    N = len(vertices)
    e_kr = np.empty(N, dtype=e_k.dtype)  # Use empty for performance
    tree = cKDTree(vertices)

    for i in range(N):
        neighbors = tree.query_ball_point(vertices[i], r=radius)
        if neighbors:
            e_kr[i] = np.mean(e_k[neighbors])
        else:
            e_kr[i] = e_k[i]  # fallback

    return e_kr


In [42]:
e_k = compute_local_bending_energy(vertices, v_normals, adj)
e_k = e_k.astype(np.float32)

radius = 0.01
e_kr = compute_integrated_bending_energy_fast(e_k, vertices, radius)

In [46]:
def compute_bending_energy_statistics(e_kr, original_indices, fractured_indices):
    e_original = e_kr[original_indices]
    e_fractured = e_kr[fractured_indices]

    stats = {
        'original_mean': np.mean(e_original),
        'original_std': np.std(e_original),
        'fractured_mean': np.mean(e_fractured),
        'fractured_std': np.std(e_fractured),
        'original_values': e_original,
        'fractured_values': e_fractured
    }
    return stats

def classify_vertices_by_threshold(e_kr, threshold):
    return e_kr > threshold  # True = fracture, False = original

def evaluate_classification(e_kr, labels, threshold):
    """
    Parameters:
    - e_kr: integrated bending energy
    - labels: array where 1=fractured, 0=original
    """
    preds = classify_vertices_by_threshold(e_kr, threshold).astype(int)
    correct = np.sum(preds == labels)
    accuracy = correct / len(labels)
    return accuracy, preds


In [45]:
def tune_radius(vertices, e_k, original_indices, fractured_indices, radius_list):
    best_radius = None
    best_accuracy = 0
    best_stats = None

    for r in radius_list:
        e_kr = compute_integrated_bending_energy_fast(e_k, vertices, r)
        stats = compute_bending_energy_statistics(e_kr, original_indices, fractured_indices)
        threshold = (stats['original_mean'] + stats['fractured_mean']) / 2

        labels = np.zeros(len(e_kr), dtype=int)
        labels[original_indices] = 0
        labels[fractured_indices] = 1

        accuracy, _ = evaluate_classification(e_kr, labels, threshold)
        print(f"Radius: {r:.4f} | Accuracy: {accuracy:.4f}")

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_radius = r
            best_stats = stats

    return best_radius, best_stats, best_accuracy


In [43]:
print(np.mean(e_kr[test_fractured_indices]))

111.09135


In [ ]:
# Create labels for training set
N = len(e_kr)
labels = np.zeros(N, dtype=int)
labels[training_fractured_indices] = 1
labels[training_original_indices] = 0


In [47]:
stats = compute_bending_energy_statistics(e_kr, original_indices, fractured_indices)

# Decision threshold = midpoint between means
threshold = (stats['original_mean'] + stats['fractured_mean']) / 2

print(f"Original mean: {stats['original_mean']:.6f}")
print(f"Fractured mean: {stats['fractured_mean']:.6f}")
print(f"Decision threshold: {threshold:.6f}")

# Predict: fractured if energy > threshold
predictions = classify_vertices_by_threshold(e_kr, threshold)  # boolean mask


Original mean: 163.092957
Fractured mean: 111.091347
Decision threshold: 137.092148


In [48]:
# Build ground-truth label array (0 = original, 1 = fractured)
labels = np.zeros_like(e_kr, dtype=int)
labels[fractured_indices] = 1
labels[original_indices] = 0

accuracy, preds = evaluate_classification(e_kr, labels, threshold)
print(f"Training classification accuracy: {accuracy:.4f}")


Training classification accuracy: 0.9070


In [ ]:
# # Get the index of the face color

# # Fractured surface
# magenta_index = colors.index('magenta')  # 4
# magenta_points = list(clusters[magenta_index]) if magenta_index < len(clusters) else []

# cyan_index = colors.index('cyan')  # 4
# cyan_points = list(clusters[cyan_index]) if cyan_index < len(clusters) else []

# # Original surface
# red_index = colors.index('red')  # 4
# red_points = list(clusters[red_index]) if red_index < len(clusters) else []

# blue_index = colors.index('blue')  # 4
# blue_points = list(clusters[blue_index]) if blue_index < len(clusters) else []



In [ ]:
# print(len(magenta_points))
# print(len(cyan_points))
# print(len(red_points))
# print(len(blue_points))

In [ ]:
# original_indices = red_points[:50] + blue_points[:50]
# fractured_indices = cyan_points[:50] + magenta_points[:50]


In [ ]:
original_values = e_kr[original_indices]
fractured_values = e_kr[fractured_indices]

import matplotlib.pyplot as plt

plt.hist(original_values, bins=30, alpha=0.6, label='Original')
plt.hist(fractured_values, bins=30, alpha=0.6, label='Fractured')
plt.axvline(x=np.mean(original_values), color='blue', linestyle='--')
plt.axvline(x=np.mean(fractured_values), color='red', linestyle='--')
plt.legend()
plt.title("Distribution of e_{k,r}(p)")
plt.xlabel("Integrated Bending Energy")
plt.ylabel("Frequency")
plt.show()

In [ ]:
# Chọn ngưỡng tự động dựa trên giá trị trung bình
threshold = (np.mean(original_values) + np.mean(fractured_values)) / 2

# Gán nhãn: 0 = original, 1 = fractured
vertex_labels = (e_kr > threshold).astype(int)


In [ ]:
face_labels = []
for face in faces_nD:
    vertex_idxs = face
    fractured_count = np.sum(vertex_labels[vertex_idxs])
    if fractured_count / len(vertex_idxs) >= 0.6:  # > 50% là gãy
        face_labels.append(1)
    else:
        face_labels.append(0)

face_labels = np.array(face_labels)


In [ ]:
# Tạo unstructured grid mới để tô màu theo mặt
segmented_mesh = pv.PolyData(vertices, smoothed_mesh.faces)
segmented_mesh.cell_data['surface_type'] = face_labels

# Hiển thị
plotter = pv.Plotter()
plotter.add_mesh(segmented_mesh, scalars='surface_type', cmap='coolwarm', show_edges=True)
plotter.add_legend([('Original', 'b'), ('Fractured', 'r')])
plotter.add_title("Surface Classification (Original vs Fractured)")
plotter.show()


### Test find_cluster_edge_points()

In [9]:
def find_cluster_edge_points(clusters, adj):
    """
    Identify edge points in each cluster based on adjacency.

    Args:
        clusters (List[Set[int]]): List of clusters, each a set of vertex indices.
        adj (Dict[int, Set[int]]): Vertex adjacency list.

    Returns:
        List[Set[int]]: List of sets, each containing the edge vertex indices for that cluster.
    """
    cluster_edge_points = []

    for cluster in clusters:
        visited = set()
        edge_points = set()
        queue = deque()

        # Initialize BFS with any point in the cluster
        if not cluster:
            cluster_edge_points.append(set())
            continue
        start = next(iter(cluster))
        queue.append(start)
        visited.add(start)

        while queue:
            vi = queue.popleft()
            for vj in adj[vi]:
                if vj not in cluster:
                    edge_points.add(vi)
                elif vj not in visited:
                    visited.add(vj)
                    queue.append(vj)

        cluster_edge_points.append(edge_points)

    return cluster_edge_points


In [10]:
edge_points_per_cluster = find_cluster_edge_points(merged, adj)
# Combine all edge points from all clusters
all_edge_points = set()
for edge_set in edge_points_per_cluster:
    all_edge_points.update(edge_set)

In [ ]:
colors = [
    'red', 'green', 'blue', 'yellow', 'magenta', 'cyan',
    'orange', 'lime', 'deeppink', 'deepskyblue', 'gold',
    'indigo', 'teal', 'crimson', 'mediumvioletred',
    'chartreuse', 'orangered', 'darkturquoise',
    'slateblue', 'darkgoldenrod'
]

plotter = pv.Plotter(shape=(1, 2))
    
plotter.subplot(0, 0)
plotter.add_title("Remove Noise Faces")
plotter.add_mesh(smoothed_mesh, show_edges=True)
for i, cluster in enumerate(merged):
    merged_indices = list(cluster)
    color = colors[i % len(colors)]
    plotter.add_mesh(smoothed_mesh.points[merged_indices], color=color, point_size=10)

plotter.subplot(0, 1)
# Extract coordinates of all edge points
plotter.add_title("Edges")
edge_coords = smoothed_mesh.points[list(all_edge_points)]
point_cloud = pv.PolyData(edge_coords)
# Plot edge points
plotter.add_mesh(point_cloud, color='red', point_size=5)


plotter.link_views()
plotter.show()

In [ ]:
# import nbformat

# # 📌 BƯỚC 1: Đặt tên file notebook và file xuất ra
# notebook_file = "pyvista_learning.ipynb"     # 👉 đổi thành tên notebook của bạn
# output_file = "extract_code_from_ipynb.py"      # 👉 tên file .py để lưu code

# # 📌 BƯỚC 2: Đọc file notebook
# with open(notebook_file, "r", encoding="utf-8") as f:
#     nb = nbformat.read(f, as_version=4)

# # 📌 BƯỚC 3: Giả sử ô hiện tại đang chạy là ô cuối cùng (có thể điều chỉnh nếu cần)
# current_cell_index = len(nb.cells) - 1

# # 📌 BƯỚC 4: Lấy code từ tất cả các cell code phía trên
# code_above = ""
# for i in range(current_cell_index):
#     cell = nb.cells[i]
#     if cell.cell_type == "code":
#         code_above += cell.source + "\n\n"

# # 📌 BƯỚC 5: Lưu vào file .py
# with open(output_file, "w", encoding="utf-8") as f:
#     f.write(code_above)

# print(f"✅ Đã lưu mã từ các cell phía trên vào file: {output_file}")
